# **LAB THREE: SELECTIVE EXCITATION**

This lab covers selective excitation, RF pulse design using Sinc and SLR, slice profiles, slice offsetting, and the relationships between pulse bandwidth, gradient strength, slice thickness and SNR.

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
LAB_USER_NAME = 'REPLACE_ME'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [2]:
import panel as pn
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import yaml
import numpy as np

from matipo import GLOBALS_DIR, DATA_DIR

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join(DATA_DIR, LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

GAMMA_BAR = 42.58e6

try:
    with open(GLOBALS_DIR+'gradient_calibration.yaml', 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
        G_CAL = 1/(GAMMA_BAR*data['gradient_calibration']) # convert
except IOError:
    print('Unable to load gradient calibration, using default value')
    G_CAL = 0.2

try:
    with open(GLOBALS_DIR+'hardpulse_90.yaml', 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
        AREA_90 = data['a_90']*data['t_90']
except IOError:
    print('Unable to load pulse calibration, using default value')
    AREA_90 = 0.3*32e-6

User data directory: /home/data/REPLACE_ME


## 1. Projection Images
Thus far, the 2D images created in the labs have been projection images. These images are created using a rectangular RF-pulse (known as a hard pulse) and both frequency and phase encoding gradients (Fig.1). The frequency and phase encoding gradients allow the aquired signals to be sorted into 2D space to produce an image. The orientation of the image will depend on which axes the frequency and phase gradients were applied along. Since there is no form of encoding in the third dimension of the sample, the signals along this axis will be summed in the 2D image.   

<center><img src="Images/projection.png" width="1300"></center>
<center><figcaption style="width: 600px;">Figure 1: Setting the frequency and phase encoding in the x and z axes produces a projection image where the varying signal components in the y direction are summed into average values.  </figcaption></center>


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Generating a Projection Image**
> 1. Insert the 3 cell phantom at the correct depth.
> 2. Choose XY for the phase encoding and frequency encoding axes and start the experiment by pressing "Run". (If the image is offset, run the Find Frequency dashboard)
> 3. **Question:** Can you tell the vertical position of the cells?
> 4. Try changing the frequency encoding axis to Z to get a side profile.
> 5. Insert the mystery phantom and repeat the XY and ZY images.
> 6. **Question:** Is the text embedded in the phantom readable?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
# Experiment 1.1: 2D RARE projection image experiment with selectable phase and frequency encoding axis

#
# Shared inputs, also used later
#
RESOLUTION = 64

G_IMAGE = 0.1 # T/m

AXIS_VECTORS = {
    'X': np.array([1,0,0], dtype=float),
    'Y': np.array([0,1,0], dtype=float),
    'Z': np.array([0,0,1], dtype=float)
}

f_axis_input = pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width = 200)
p_axis_input = pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200)

def get_g_phase():
    g_phase_max = G_IMAGE/G_CAL
    g_phase_prop = np.linspace(1, -1, RESOLUTION, endpoint=False)
    return np.outer(g_phase_max*g_phase_prop, AXIS_VECTORS[p_axis_input.value])

def get_g_read():
    g_read_max = G_IMAGE/G_CAL
    return g_read_max*AXIS_VECTORS[f_axis_input.value]

#
# Experiment 1
#

import importlib
import RARE2D
importlib.reload(RARE2D)

from RARE2D import RARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=get_g_read,
    g_phase_1=get_g_phase,
    phase_order=range(RESOLUTION),
    n_ETL=RESOLUTION,
    t_echo=0.01,
    n_scans=2,
    n_samples=RESOLUTION,
    t_dw=20e-6,
    t_end=0.5
)

override_pars['t_read'] = override_pars['t_dw']*override_pars['n_samples']
override_pars['t_phase'] = override_pars['t_read']/2

# create dashboard app
exp1_app = RARE2DApp(
    override_pars=override_pars
)

exp1_app.plot1.figure.height=400
exp1_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>2D Projection Image</h2></center>', sizing_mode='stretch_width'),
    pn.Row(pn.pane.HTML('<label>Frequency Encoding Axis</label>', width=200), f_axis_input),
    pn.Row(pn.pane.HTML('<label>Phase Encoding Axis</label>', width=200), p_axis_input), 
    exp1_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width=200)
    [2] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width=200)
    [3] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

As you can see, projection images aren't always useful because summing a large amount of signal in the object's third dimension makes it almost impossible to see details that are only present in specific parts of the sample. If we want to see detail in a specific portion of the object we need to be able to limit the amount of signal that is being summed in one direction. This is done through slice selection.

## 2. Frequency Selective Excitation

In order to image a single slice rather than the entire sample, two main changes need to be made to the pulse sequence:

1) A slice-select gradient must be added along the axis perpendicular to the plane of the desired slice. This causes the spins along the chosen axis to precess at different frequencies making it possible to selectively excite them by transmitting at a specific frequency.

2) The RF excitation pulse must be modified to only transmit a small range of frequencies. So far, we have used short (~20 us) high amplitude pulses which excite a very broad range of frequencies (called a "hard pulse"). As the pulse width is increased, the range of frequencies contained in it (the bandwidth) decreases, so we need to increase the pulse length to achieve a narrow slice. Flip angle is proportional to the area of the pulse, so a long, narrow-bandwidth pulse needs less amplitude to achieve the same flip angle. This is called a "soft pulse".

The resulting pulse sequence is shown in Figure 2 below. 

<center><img src="Images/slice.png" width="1000"></center>
<center><figcaption style="width: 600px;">Figure 2: Slicing along the z axis allows the sample to be imaged at a specific cross section index     </figcaption></center>


The relationship of the slice thickness ($\Delta$z) to bandwidth ($\Delta$F) and slice-select gradient strength ($G_{ss}$) is governed by the following equation:

$$\Delta z = \frac{\Delta F}{\bar\gamma \cdot G_{ss}} \tag{1}$$

where $\bar\gamma$ is the gyromagnetic ratio (for hydrogen nucleus $\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$). Figure 3 shows examples of how slice thickness can be controlled by varying $\Delta F$ or $G_{ss}$.

<center><img src="Images/control_slice_thickness.png" width="800"></center>
<center><figcaption style="width: 800px;">Figure 3: How the slice thickness is related to the RF pulse bandwidth and slice-select gradient strength </figcaption></center>

In the next experiment, we will use a simplified pulse sequence (Fig.4 below) that performs a spin echo with a constant gradient present during both the RF excitation pulse and the readout. This allows the sample to be selectively excited and then imaged along the same axis to directly measure the excitation profile.

<center><img src="Images/const_grad.png" width=600"></center>
<center><figcaption style="width: 600px;">Figure 4: Pulse Sequence for the Constant Gradient Spin Echo experiment</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1:**
> 1. Insert the Shim Sample and run the below experiment with a pulse width of 20 us. You should get a 1D Z projection image of the sample.
> 2. Increase the pulse width and rerun the experiment. Try with a few different pulse widths.
> 3. Note: With a short RF pulse, there should be roughly equal signal from all parts of the sample along the Z axis.
> 4. **Question:** With a long RF pulse, do you still get equal signal from the whole sample? If not, which region dominates?
> 5. **Question:** How does the Z thickness of the dominant region (slice thickness) vary with the pulse width?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
# Experiment 2
import importlib
import SE_const_grad_app
importlib.reload(SE_const_grad_app)

from SE_const_grad_app import ConstGradSEApp # from dashboards-inline directory that was added to sys.path

exp2_width_input = pn.widgets.FloatInput(name="pulse width (μs)", start=20, end=1000, step=100, value=20, width=80)

# set some parameters directly
override_pars = dict(
    a_90=lambda: 0.5*AREA_90/(exp2_width_input.value*1e-6),
    t_90=lambda: exp2_width_input.value*1e-6,
    g_read=(0,0,0.05/G_CAL),
    n_scans=2,
    n_samples=200,
    t_dw=40e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5,
    t_echo = 10e-3
)

# create dashboard app
exp2_app = ConstGradSEApp(
    override_pars=override_pars,
    show_magnitude=False,
    show_complex=True,
    # enable_run_loop=True,
    flat_filter = True
)

exp2_app.plot1.figure.height=400
exp2_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>Constant Gradient Spin Echo</h2></center>', sizing_mode='stretch_width'),
    exp2_width_input,
    exp2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] FloatInput(end=1000, name='pulse width (μs)', start=20, step=100, value=20, width=80)
    [2] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.2: 2D RARE Using Rectangular Soft Pulse**
> 1. Insert the three cell phantom, and run the experiment with the pulse width set to 20 us and the axes set to X-Y to get a projection image.
> 2. Try increasing the pulse width to selectively excite only the middle cell. You can also switch one of the axes to Z to visualise the excitation profile as in the previous experiment.
> 3. **Question**: Approximately how long does the RF pulse need to be to only get noticable signal from one cell?
> 4. **Question**: What is the thickness of this slice? (You can check with the Constant Gradient Spin Echo experiment above)
> 5. **Question**: If you continue increasing the pulse width, how is the SNR of the image related to the slice thickness?**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
# Experiment 3: 2D RARE experiment with slice gradient and square slice pulse

import importlib
import custom_pulse_RARE2D
importlib.reload(custom_pulse_RARE2D)

from custom_pulse_RARE2D import CustomPulseRARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    a_90=lambda: 1.0*AREA_90/(exp2_width_input.value*1e-6),
    t_90=lambda: exp2_width_input.value*1e-6,
    shape_90=[1],
    g_slice=(0,0,0.05/G_CAL),
    g_read=get_g_read,
    g_phase_1=get_g_phase,
    phase_order=range(RESOLUTION),
    n_ETL=RESOLUTION,
    n_scans=4,
    n_samples=RESOLUTION,
    t_dw=20e-6,
    t_end=0.5,
    t_grad_ramp=50e-6, # TODO: fix strange artifacts with larger t_grad_ramp
    n_grad_ramp=5
)

override_pars['t_read'] = override_pars['t_dw']*override_pars['n_samples']
override_pars['t_phase'] = override_pars['t_read']/2

# create dashboard app
exp3_app = CustomPulseRARE2DApp(
    override_pars=override_pars
)

exp3_app.plot1.figure.height=400
exp3_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>2D Image with Rectangular RF Pulse and Slice Select Gradient</h2></center>', sizing_mode='stretch_width'),
    exp2_width_input,
    pn.Row(pn.pane.HTML('<label>Frequency Encoding Axis</label>', width=200), f_axis_input),
    pn.Row(pn.pane.HTML('<label>Phase Encoding Axis</label>', width=200), p_axis_input), 
    exp3_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] FloatInput(end=1000, name='pulse width (μs)', start=20, step=100, value=20, width=80)
    [2] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width=200)
    [3] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width=200)
    [4] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 3. Soft Pulse Design

In practice, a slice will have some finite thickness. Choosing a slice thickness is a trade off between two main factors: A thicker slice will have more signal because there are more nuclear spins available for excitation; a thinner slice will have less averaging over the slice axis resulting in better selection of individual features in the sample. In the previous experiments, a rectangular RF pulse with a constant amplitude was used. This resulted in a Sinc function shaped excitation profile (or slice profile). The side-lobes of the slice profile partially excite the top and bottom cells causing them to show faintly in the image. 

For small tip angles ($\alpha \ll 90^\circ$), the small-tip-angle approximation states that the signal spectrum will be approximately proportional to the spectrum of the RF pulse, and therefore will have the same shape. If the tip angle at any point in the sample approaches or exceeds $90^\circ$ this approximation breaks down and the signal spectrum will be distorted, compared to the RF pulse spectrum, due to the sine relationship between signal and tip angle (Fig. 5).

<center><img src="Images/tip_angle.png" width="900"></center>
<center><figcaption style="width: 500px;">Figure 5: How the RF pulse shape relates to the NMR signal</figcaption></center>

To perfectly isolate the middle cell, it should be fully excited (90 degree tip angle throughout) while the outer cells are not excited at all (0 degree tip angle). This requires the frequency spectrum of the RF pulse to have a rectangular shape, which can be theoretically achieved with a Sinc shaped RF pulse amplitude modulation as shown in Figure 6:

<center><img src="Images/soft_pulse_parameters.png" width="600"></center>
<center><figcaption style="width: 600px;">Figure 6: An ideal RF pulse creates a rectangular slice select profile</figcaption></center>

In practice, the theoretical Sinc function is infinitely long, and must be truncated to be used. This may be done at the zero crossings to minimise ringing artifacts (Gibbs phenomenon). Additionally, the Sinc shape can be multiplied with a gaussian to further reduce the Gibbs phenomenon. This process is called Apodization.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1: Visualising Sinc Pulse Design**
>The experiment below is designed to help you visualise how changing the parameters of a soft pulse changes the profile in the frequency domain.The RF pulse you design will be used for the following experiments.
> 1. Experiment with the three soft pulse parameters and observe how the shape of the RF-pulse and slice profile changes. 
> 2. **Question:** How does changing the lobe width affect the spectral bandwidth?
> 3. **Question:** How does adding more lobes improve the shape of the slice profile? What is the trade-off regarding the overall pulse width?
> 4. **Question:** What happens when apodization is too large? Pick a value that looks like a good compromise.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
# Experiment 4.1

from matipo.util.pulseshape import calc_soft_pulse
from matipo.util.plots import SharedXPlot, ComplexPlot
from scipy.fft import fft,fftfreq, fftshift, ifftshift
from bokeh.plotting import figure

exp4_lobe_width_input = pn.widgets.FloatInput(name="lobe width (μs)", start=50, end=400, step=50, value=100, width=80)
exp4_n_lobe_input = pn.widgets.FloatInput(name="lobes", start=0, end=10, step=1, value=3, width=80)
exp4_apodization_input = pn.widgets.FloatInput(name="apodization", start=0, end=100, step=1, value=0, width=80)

exp4_p1 = ComplexPlot(
    title="Waveform",
    x_axis_label="Time (s)",
    y_axis_label="Amplitude",
    height=400)

exp4_p2 = ComplexPlot(
    title="Spectrum",
    x_axis_label="Frequency (Hz)",
    y_axis_label="Spectral Density",
    height=400)

# auto frequency range based on max bandwidth
exp4_p2.figure.x_range.start = -4/(1e-6*exp4_lobe_width_input.start)
exp4_p2.figure.x_range.end = 4/(1e-6*exp4_lobe_width_input.start)

plot_row = pn.Row(exp4_p1.figure, exp4_p2.figure, sizing_mode='stretch_width')

global_shape = [1]
global_width = 100e-6

def update_plots(event):
    global global_width, global_shape 
    width = max(1, exp4_n_lobe_input.value)*exp4_lobe_width_input.value*1e-6 # max(1, ...) to prevent width=0 when n_lobes=0
    N = int(width / 1e-6)
    if N > 1000:
        N = 1000
    dt = width / N
    pts = np.sinc(np.linspace(-exp4_n_lobe_input.value, exp4_n_lobe_input.value,N)) 
    t = np.arange(N)*dt
    t_0 = (width-dt)/2
    pts*=np.exp(-exp4_apodization_input.value*(np.linspace(-1, 1, N)**2))
    global_shape = pts
    global_width = width
    freq = fftfreq(N,dt)
    spectrum = fft(pts)*dt
    spectrum *= np.exp(-1j*np.pi*(width+dt)*freq) # fix phase of spectrum plot due to time offset
    freq = fftshift(freq)
    spectrum = fftshift(spectrum)
    exp4_p1.update_data(t, pts)
    exp4_p2.update_data(freq, spectrum)
    pn.io.push_notebook(plot_row)

# update plot when any value is changed
exp4_lobe_width_input.param.watch(update_plots, 'value')
exp4_n_lobe_input.param.watch(update_plots, 'value')
exp4_apodization_input.param.watch(update_plots, 'value')

# manually trigger for first update
exp4_lobe_width_input.param.trigger('value')

app = pn.Column(
    pn.pane.HTML('<center><h2>Sinc RF Pulse Design</h2></center>', sizing_mode='stretch_width'),
    pn.Row(
        pn.Column(exp4_lobe_width_input, exp4_n_lobe_input, exp4_apodization_input),
        plot_row,
        sizing_mode='stretch_width'),
    sizing_mode='stretch_width')

app

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] Column
            [0] FloatInput(end=400, name='lobe width (μs)', start=50, step=50, value=100, width=80)
            [1] FloatInput(end=10, name='lobes', start=0, step=1, value=3, width=80)
            [2] FloatInput(end=100, name='apodization', start=0, step=1, width=80)
        [1] Row(sizing_mode='stretch_width')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.2: 1D Slice Profile**
> With the Sinc pulse designed, we can now use it to create a 1D image of the slice profile.
> You will need Equation 1: $$\Delta z = \frac{\Delta F}{\bar\gamma \cdot G_{ss}}$$ where $\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$ for the hydrogen nucleus.
> 1. Based on the bandwidth ($\Delta$F) of the Sinc pulse you have designed and the value of the slice gradient ($G_{ss}$) shown below, calculate the expected thickness of the slice profile.
> 2. Insert the shim sample (make sure to centre the sample using the depth gauge) and run the experiment.
> 3. **Question:** How does the slice profile compare to the calculated frequency spectrum?
> 4. Try changing the flip angle to 30 and 180 and compare the resulting spectrum.
> 5. **Question:** At what flip angle (out of 30, 90, or 180) does the signal spectrum look most similar to the RF pulse spectrum?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Experiment 4.2

import importlib
import custom_pulse_SE_app
importlib.reload(custom_pulse_SE_app)

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    return min(1, abs(AREA_90/shape_area))

exp4p2_GSS_input = pn.widgets.FloatInput(name="Slice Select Gradient (mT/m)", start=0, end=150, step=10, value=100, width=80, disabled=False)
exp4p2_flipangle_input = pn.widgets.FloatInput(name="Target Flip Angle (degrees)", start=10, end=360, step=10, value=90, width=80, disabled=False)

from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

# set some parameters directly
override_pars = dict(
    a_90=lambda: (exp4p2_flipangle_input.value/90)*estimate_a_90(),
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=lambda: (0,0,-1e-3*exp4p2_GSS_input.value/G_CAL),
    g_read=(0,0,G_IMAGE/G_CAL),
    n_scans=2,
    n_samples=200,
    t_dw=20e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp4p2_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp4p2_app.plot1.figure.height=400
exp4p2_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>Sinc RF Pulse Slice Profile Measurement</h2></center>', sizing_mode='stretch_width'),
    exp4p2_GSS_input,
    exp4p2_flipangle_input,
    exp4p2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] FloatInput(end=150, name='Slice Select G..., start=0, step=10, value=100, width=80)
    [2] FloatInput(end=360, name='Target Flip A..., start=10, step=10, value=90, width=80)
    [3] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

### Note: Phase Rewinding Gradients

There may be some twisting in the imaginary components of the spectrum in the previous experiment. This is due to dephasing. The application of the slice-select gradient during the soft pulse causes the spins to accumulate different amounts of phase based on their position along the the gradient's axis. This phase dispersion of the transverse magnetization will result in a loss of signal when averaged together. Since the dephasing that occurs is linear, the phase effects can be cancelled by applying a gradient in the opposite direction. This is done using a rephasing gradient. In general, the area of the rephasing gradient is half that of the slice-select gradient. This is done under the assumption that the majority of the spins are tipped into the transverse plane at the centre of the 90-degree pulse and, therefore, phase dispersion primarily occurs during the second half of the slice select gradient. While this assumption is a good starting point, the size of the refocusing gradient needs to be calculated based on the soft pulse design if the phase effects are to be cancelled perfectly.

<center><img src="Images/refocusing_gradient.png" width="800"></center>
<center><figcaption style="width: 600px;">Figure 7: Slice Select Spin Echo pulse sequence without (left) and with (right) rephasing lobes</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.3: 2D RARE Using Sinc Shaped Soft Pulse**
> 1. Insert the three cell phantom, and run the experiment below with the Z-Y axes selected.
> 2. Adjust the slice select gradient to change the slice thickness until only the middle cell is visible. You may need to also modify the sinc pulse design above if the gradient is reaching the maximum allowed. The sample can also be moved up or down if it is not centred.
> 3. Change the axes to X-Y and run again. You should see only the middle cell appearing in the image. If not, go back and adjust the soft pulse/gradient parameters until you do.
> 4. **Save/screenshot the image**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
# Experiment 5: 2D RARE experiment with slice gradient and sinc slice pulse

import importlib
import custom_pulse_RARE2D
importlib.reload(custom_pulse_RARE2D)

from custom_pulse_RARE2D import CustomPulseRARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    a_90=estimate_a_90,
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=lambda: (0,0,-1e-3*exp4p2_GSS_input.value/G_CAL),
    g_read=get_g_read,
    g_phase_1=get_g_phase,
    n_ETL=RESOLUTION,
    t_echo=0.01,
    n_scans=4,
    n_samples=RESOLUTION,
    t_dw=20e-6,
    t_end=0.5,
)

override_pars['t_read'] = override_pars['t_dw']*override_pars['n_samples']
override_pars['t_phase'] = override_pars['t_read']/2

# create dashboard app
exp5_app = CustomPulseRARE2DApp(
    override_pars=override_pars
)

exp5_app.plot1.figure.height=400
exp5_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>2D Slice Image Using Sinc RF Pulse</h2></center>', sizing_mode='stretch_width'),
    exp4p2_GSS_input,
    pn.Row(pn.pane.HTML('<label>Frequency Encoding Axis</label>', width=200), f_axis_input),
    pn.Row(pn.pane.HTML('<label>Phase Encoding Axis</label>', width=200), p_axis_input), 
    exp5_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] FloatInput(end=150, name='Slice Select G..., start=0, step=10, value=100, width=80)
    [2] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width=200)
    [3] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width=200)
    [4] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 5. SLR Soft Pulse Design

The Sinc method of designing a slice select RF pulse works well at small flip angles, but is not ideal for 90 degree pulses and is quite poor for 180 pulses. A more optimal method, that is widely used, is the Shinnar-Le Roux algorithm. This algorithm takes into account the non-linearity of NMR excitation to achieve equal ripple in the final magnetization spectrum and, therefore, signal spectrum.

For 90 degree pulses, this method results in an RF pulse spectrum with minimal sidelobes at the expense of poor flatness on top. However, since the signal level resulting from a near 90 degree flip angle pulse is not very sensitive to the exact flip angle, the slice profile is still quite flat.

The SLR algorithm is also used to design asymmetrical RF pulses which are used for advanced techniques. 

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 5.1: Pulse Design For a Target Slice Thickness Using SLR**
> You will need Equation 1: $$\Delta z = \frac{\Delta F}{\bar\gamma \cdot G_{ss}}$$ where $\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$ for the hydrogen nucleus.
> 1. **Question**: Calculate the required bandwidth to achieve a 1 mm slice with the maximum slice gradient, 150 mT/m, and enter the value below (take care with units).
> 2. Insert the Shim sample.
> 3. Measure the slice profile with the above slice select gradient and check the thickness is correct. You may increase the pulse duration to improve the sharpness of the edges of the slice.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
# Experiment 6
from matipo.util.pulseshape import calc_soft_pulse
from matipo.util.plots import SharedXPlot, ComplexPlot
from scipy.fft import fft,fftfreq, fftshift, ifftshift
from bokeh.plotting import figure

exp6_width_input = pn.widgets.FloatInput(name="duration (µs)", start=100, end=1000, step=10, value=300, width=80)
exp6_bandwidth_input = pn.widgets.FloatInput(name="bandwidth (kHz)", start=4, end=40, step=1, value=20, width=80)

exp6_p1 = ComplexPlot(
    title="Waveform",
    x_axis_label="Time (s)",
    y_axis_label="Amplitude",
    height=400)

exp6_p2 = ComplexPlot(
    title="Spectrum",
    x_axis_label="Frequency (Hz)",
    y_axis_label="Spectral Density",
    height=400)

# auto frequency plot x range based on max bandwidth
exp6_p2.figure.x_range.start = -2*1e3*exp6_bandwidth_input.end
exp6_p2.figure.x_range.end = 2*1e3*exp6_bandwidth_input.end

exp6_plot_row = pn.Row(exp6_p1.figure, exp6_p2.figure, sizing_mode='stretch_width')

global_slr_shape = [1]
global_slr_width = 100e-6

def update_plots(event):
    global global_slr_width, global_slr_shape
    # if pulse width*bandwidth is too small, force pulse width to increase
    if 1e-6*exp6_width_input.value*1e3*exp6_bandwidth_input.value < 1.44:
        exp6_width_input.value = 1e3*1.44/exp6_bandwidth_input.value
    bw = 1e3*exp6_bandwidth_input.value
    N, dt, pts = calc_soft_pulse(1e-6*exp6_width_input.value, bw)
    global_slr_width = 1e-6*exp6_width_input.value
    global_slr_shape = pts
    t = np.arange(N)*dt
    spectrum = fft(fftshift(pts))
    freq = fftfreq(N,dt)
    freq = fftshift(freq)
    spectrum = fftshift(spectrum)
    exp6_p1.update_data(t, pts)
    exp6_p2.update_data(freq, spectrum)
    pn.io.push_notebook(exp6_plot_row)

# update plot when any value is changed
exp6_width_input.param.watch(update_plots, 'value')
exp6_bandwidth_input.param.watch(update_plots, 'value')

# manually trigger for first update
exp6_width_input.param.trigger('value')

exp6_app = pn.Column(
    pn.pane.HTML('<center><h2>SLR-based RF Pulse Design</h2></center>', sizing_mode='stretch_width'),
    pn.Row(
        pn.Column(exp6_width_input, exp6_bandwidth_input),
        exp6_plot_row,
        sizing_mode='stretch_width'),
    sizing_mode='stretch_width')

exp6_app

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] Column
            [0] FloatInput(end=1000, name='duration (µs)', start=100, step=10, value=300, width=80)
            [1] FloatInput(end=40, name='bandwidth (kHz)', start=4, step=1, value=20, width=80)
        [1] Row(sizing_mode='stretch_width')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)

In [10]:
# Experiment 6.2

import importlib
import custom_pulse_SE_app
importlib.reload(custom_pulse_SE_app)

from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

# set some parameters directly
override_pars = dict(
    a_90=lambda: min(1, abs(AREA_90/(np.mean(global_slr_shape)*global_slr_width))),
    t_90=lambda: global_slr_width,
    shape_90=lambda: global_slr_shape,
    g_slice=lambda: (0,0,-1e-3*exp4p2_GSS_input.value/G_CAL),
    g_read=(0,0,G_IMAGE/G_CAL),
    n_scans=2,
    n_samples=200,
    t_dw=20e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp6_2_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp6_2_app.plot1.figure.height=400
exp6_2_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>SLR RF Pulse Slice Profile Measurement</h2></center>', sizing_mode='stretch_width'),
    exp4p2_GSS_input,
    exp6_2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] FloatInput(end=150, name='Slice Select G..., start=0, step=10, value=100, width=80)
    [2] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 6. Controlling Slice Position

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 6.1: Locating the Hidden Text**
>1. Insert the mystery phantom and adjust the settings to get an XY image of a slice through the text inside the phantom.
>2. **Hint:** Try reducing the slice select gradient to get a thicker slice and viewing a Z-Y projection to identify the plane where the text is. You can measure the Z offset of the text and calculate the required frequency offset to position the slice correctly.
> 3. **Note:** the image has some upscaling postprocessing to make it easier to see the details.
> 4. **Save/screenshot the image**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
# Experiment 7: 2D RARE experiment with slice gradient and SLR slice pulse

import importlib
import custom_pulse_RARE2D
importlib.reload(custom_pulse_RARE2D)

exp7_freqoffset_input = pn.widgets.FloatInput(name="Pulse Freq. Offset (kHz)", start=-30, end=30, step=1, value=0, width=80)

from custom_pulse_RARE2D import CustomPulseRARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    a_90=lambda: min(1, abs(AREA_90/(np.mean(global_slr_shape)*global_slr_width))),
    t_90=lambda: global_slr_width,
    shape_90=lambda: global_slr_shape,
    f_slice_offset=lambda: 1e3*exp7_freqoffset_input.value,
    g_slice=lambda: (0,0,1e-3*exp4p2_GSS_input.value/G_CAL),
    g_read=get_g_read,
    g_phase_1=get_g_phase,
    n_ETL=RESOLUTION,
    n_scans=16,
    n_samples=RESOLUTION,
    t_dw=20e-6,
    t_end=0.5
)

override_pars['t_read'] = override_pars['t_dw']*override_pars['n_samples']
override_pars['t_phase'] = override_pars['t_read']/2

# create dashboard app
exp7_app = CustomPulseRARE2DApp(
    override_pars=override_pars,
    upscale_factor=2,
    gaussian_blur=1
)

# display layout
pn.Column(
    pn.pane.HTML('<center><h2>2D Slice Image with Offsetting Using SLR RF Pulse</h2></center>', sizing_mode='stretch_width'),
    exp4p2_GSS_input,
    exp7_freqoffset_input,
    pn.Row(pn.pane.HTML('<label>Frequency Encoding Axis</label>', width=200), f_axis_input),
    pn.Row(pn.pane.HTML('<label>Phase Encoding Axis</label>', width=200), p_axis_input), 
    exp7_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] FloatInput(end=150, name='Slice Select G..., start=0, step=10, width=80)
    [2] FloatInput(end=30, name='Pulse Freq. O..., start=-30, step=1, width=80)
    [3] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width=200)
    [4] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width=200)
    [5] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')